<a href="https://colab.research.google.com/github/yunjichu/image/blob/main/image_project(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정
- colab 파일에 첨부된 js파일 옮기기

In [1]:
#yt-dlp 설치
!pip install yt-dlp opencv-python-headless

# 패키지 목록 업데이트
!apt-get update

# FFmpeg 설치
!apt-get install ffmpeg -y

#opencv 설치
!pip install opencv-python-headless

#google cloud vision 설치
!pip install google-cloud-vision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,514 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get

# 유튜브에서 영상 다운 받아 1초 간격으로 프레임 추출

In [2]:
import os
import cv2
import numpy as np
from yt_dlp import YoutubeDL


# 유튜브 동영상 다운로드 함수
def download_youtube_video(url, save_path="downloaded_video.mp4"):
    """
    Downloads a YouTube video using yt-dlp.
    """
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',  # 최고 품질로 다운로드
        'outtmpl': save_path,  # 저장 파일 경로
        'noplaylist': True,    # 재생 목록 제외
    }
    with YoutubeDL(ydl_opts) as ydl:
        print(f"Downloading video from: {url}")
        ydl.download([url])
    print(f"Video downloaded to: {save_path}")

# YouTube URL 및 다운로드 경로 설정
youtube_url = "https://www.youtube.com/watch?v=BTLQXrw4ZlE"
video_path = "downloaded_video.mp4"

# 동영상 다운로드 실행
download_youtube_video(youtube_url, save_path=video_path)

!yt-dlp -f best "https://www.youtube.com/watch?v=BTLQXrw4ZlE" -o "downloaded_video.mp4"

# 기존 코드 유지: 다운로드 확인 및 FFmpeg 설치
import os
if os.path.exists("downloaded_video.mp4"):
    print("File downloaded successfully.")
else:
    print("File download failed.")

!apt-get update
!apt-get install ffmpeg -y
!ffmpeg -i downloaded_video.mp4 -vcodec libx264 fixed_video.mp4

# 새로운 코드: 프레임 추출
def get_video_fps(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return fps

def enhance_image(image):
    blur = cv2.GaussianBlur(image, (0, 0), 3)
    sharpened = cv2.addWeighted(image, 1.5, blur, -0.5, 0)
    return sharpened

def is_frame_changed(prev_frame, current_frame, threshold=30):
    if prev_frame is None:
        return True
    diff = cv2.absdiff(prev_frame, current_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count > threshold

def extract_all_frames(video_path, output_dir):
    import cv2
    """
    Extracts all frames from a video file and saves them as images.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    fps = get_video_fps(video_path)  # FPS 가져오기
    frame_number = 0
    prev_frame = None
    saved_frames = 0  # 저장된 프레임 수를 카운트

    success, image = cap.read()
    if not success:
        print("Error: Unable to read the video file.")
        return

    while success:
        if frame_number % int(fps) == 0:  # 1초 간격으로 프레임 추출
            if is_frame_changed(prev_frame, image):
                enhanced_frame = enhance_image(image)  # 품질 개선
                frame_filename = os.path.join(output_dir, f"frame_{frame_number:05d}.jpg")
                cv2.imwrite(frame_filename, enhanced_frame)
                saved_frames += 1  # 저장된 프레임 수 증가
                print(f"Saved frame: {frame_filename}")
            prev_frame = image

        success, image = cap.read()
        frame_number += 1

    cap.release()
    print(f"All frames saved to: {output_dir}")
    print(f"Total frames extracted: {saved_frames}")  # 총 저장된 프레임 수 출력

# 프레임 추출 실행
video_path = "fixed_video.mp4"  # 동영상 파일 경로
output_dir = "all_frames"      # 저장 폴더 이름

extract_all_frames(video_path, output_dir)


[youtube] Extracting URL: https://www.youtube.com/watch?v=BTLQXrw4ZlE
[youtube] BTLQXrw4ZlE: Downloading webpage
[youtube] BTLQXrw4ZlE: Downloading ios player API JSON
[youtube] BTLQXrw4ZlE: Downloading mweb player API JSON
[youtube] BTLQXrw4ZlE: Downloading player b46bb280
[youtube] BTLQXrw4ZlE: Downloading m3u8 information
[info] BTLQXrw4ZlE: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 160
[download] Destination: downloaded_video.mp4.f616.mp4
[download] 100% of  366.65MiB in 00:02:15 at 2.70MiB/s                   
[download] Destination: downloaded_video.mp4.f251.webm
[download] 100% of   13.25MiB in 00:00:01 at 7.55MiB/s   
[Merger] Merging formats into "downloaded_video.mp4.webm"
Deleting original file downloaded_video.mp4.f616.mp4 (pass -k to keep)
Deleting original file downloaded_video.mp4.f251.webm (pass -k to keep)
Video downloaded to: downloaded_video.mp4
         To let yt-dlp download and merge the best available form

# 자막 추출
- google-vision-api 사용

In [3]:
import os
import cv2
from google.cloud import vision

# 환경변수 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/image-443101-514bb45e53e7.json"
client = vision.ImageAnnotatorClient()

# OCR을 통한 텍스트 추출 함수
all_frame_dir = "/content/all_frames"
output_dir = "/content/output_texts"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_text_file = os.path.join(output_dir, "extracted_texts.txt")

image_files = [f for f in os.listdir(all_frame_dir) if f.endswith(".jpg")]
image_files.sort()

with open(output_text_file, "w") as f:
    f.write("### Vision API 추출 결과 ###\n\n")

for image_file in image_files:
    image_path = os.path.join(all_frame_dir, image_file)
    print(f"Processing: {image_path}")

    try:
        # 이미지 읽기
        image = cv2.imread(image_path)

        # OpenCV로 이미지 인코딩
        success, encoded_image = cv2.imencode(".jpg", image)
        if not success:
            print(f"Failed to encode image: {image_file}")
            continue

        # Vision API 요청 준비 및 텍스트 추출
        content = encoded_image.tobytes()
        image_vision = vision.Image(content=content)
        response = client.text_detection(image=image_vision)

        texts = response.text_annotations
        extracted_text = texts[0].description if texts else "No text detected."
        print(f"Extracted text:\n{extracted_text}")

        # 개별 텍스트 파일 저장 및 통합 텍스트 파일에 추가
        individual_text_file = os.path.join(output_dir, f"{image_file.split('.')[0]}.txt")
        with open(individual_text_file, "w") as f:
            f.write(extracted_text)

        with open(output_text_file, "a") as f:
            f.write(f"### {image_file} ###\n{extracted_text}\n\n")

    except Exception as e:
        print(f"Error processing {image_file}: {str(e)}")
        continue

print(f"All text extracted and saved to {output_text_file}.")


Processing: /content/all_frames/frame_00000.jpg
Extracted text:
7월 26일
먹태는 유치원가고 유준이는 낮잠자는중
Processing: /content/all_frames/frame_00029.jpg
Extracted text:
7월 26일
먹태는 유치원 가고 유준이는 낮잠자는중
Processing: /content/all_frames/frame_00058.jpg
Extracted text:
7월 26일
먹태는 유치원 가고 유준이는 낮잠자는 중
Processing: /content/all_frames/frame_00087.jpg
Extracted text:
(아빠 온지도 모르고 세상 편하게 자는 중 ㅋㅋ)
Processing: /content/all_frames/frame_00116.jpg
Extracted text:
(아빠 온지도 모르고 세상 편하게 자는중ㅋㅋ
Processing: /content/all_frames/frame_00145.jpg
Extracted text:
(아빠 온지도 모르고 세상 편하게 자는중ㅋㅋ
Processing: /content/all_frames/frame_00174.jpg
Extracted text:
일어날때가 됐다고 생각중이었는데 바로 잠깼어요ㅋㅋ
Processing: /content/all_frames/frame_00203.jpg
Extracted text:
일어날때가 됐다고 생각중이었는데 바로 잠깼어요 ㅋㅋ
Processing: /content/all_frames/frame_00232.jpg
Extracted text:
일어날 때가 됐다고 생각중이었는데 바로 잠깼어요 ㅋㅋ
Processing: /content/all_frames/frame_00261.jpg
Extracted text:
일어날 때가 됐다고 생각중이었는데 바로 잠깼어요ㅋㅋ
Processing: /content/all_frames/frame_00290.jpg
Extracted text:
일어날 때가 됐다고 생각중이

### **추출된 텍스트를 리스트에 튜플형식으로 저장해서 일치하는 검색어를 이용해 유튜브 타임스탬프 생성**

In [4]:
import os
import cv2

# FPS 추출 함수
def get_video_fps(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return fps

# 파일 경로와 FPS 설정
video_path = "downloaded_video.mp4"
fps = get_video_fps(video_path)

# Frame 번호에서 시간 계산 함수
def frame_to_time(frame_number, fps):
    seconds = int(frame_number // fps)  # 초를 정수로 변환
    minutes = seconds // 60
    seconds %= 60
    return f"{minutes}:{seconds:02d}"  # 형식: MM:SS

# 텍스트와 시간 추출 함수
def extract_text_and_times(file_path, fps):
    text_time_list = []

    with open(file_path, "r") as f:
        lines = f.readlines()

    current_frame = None
    current_text = []
    for line in lines:
        line = line.strip()
        if line.startswith("### frame_") and line.endswith(".jpg ###"):
            if current_frame is not None and current_text:
                timestamp = frame_to_time(current_frame, fps)
                for text in current_text:
                    text_time_list.append((text, timestamp))
            current_frame = int(line.split("frame_")[1].split(".")[0])
            current_text = []
        elif line != "텍스트를 감지하지 못했습니다.":
            if current_frame is not None:
                current_text.append(line)

    if current_frame is not None and current_text:
        timestamp = frame_to_time(current_frame, fps)
        for text in current_text:
            text_time_list.append((text, timestamp))

    return text_time_list

# 검색 텍스트로 타임스탬프 링크 생성
def search_text_and_generate_links(text_time_list, search_text, youtube_url):
    base_url = youtube_url.split("&")[0]
    links = []

    for text, timestamp in text_time_list:
        if text == search_text:
            minutes, seconds = map(int, timestamp.split(":"))
            total_seconds = minutes * 60 + seconds
            links.append(total_seconds)

    return links

# 타임스탬프 그룹화 함수
def group_timestamps(links):
    if not links:
        return []

    grouped_links = []
    links.sort()

    current_group = [links[0]]
    for i in range(1, len(links)):
        if links[i] - current_group[-1] <= 1:
            current_group.append(links[i])
        else:
            grouped_links.append(min(current_group))
            current_group = [links[i]]

    if current_group:
        grouped_links.append(min(current_group))

    return grouped_links

# 실행 예시
extracted_text_file = "/content/output_texts/extracted_texts.txt"
youtube_url = "https://www.youtube.com/watch?v=BTLQXrw4ZlE"

text_time_list = extract_text_and_times(extracted_text_file, fps)
search_text = "유산균 먹고 난 후부터는 하루에 두 번씩 쾌변 중"
links = search_text_and_generate_links(text_time_list, search_text, youtube_url)
grouped_links = group_timestamps(links)

if grouped_links:
    print(f"검색어 '{search_text}'에 해당하는 타임스탬프 링크:")
    for link in grouped_links:
        print(f"{youtube_url}&t={link}s")
else:
    print("일치하는 검색어가 없습니다.")


검색어 '유산균 먹고 난 후부터는 하루에 두 번씩 쾌변 중'에 해당하는 타임스탬프 링크:
https://www.youtube.com/watch?v=BTLQXrw4ZlE&t=809s
https://www.youtube.com/watch?v=BTLQXrw4ZlE&t=811s
